In [26]:
#TDM 예제
docs=["나는 양념 치킨을 좋아해 하지만 후라이드 치킨을 싫어해",
     "나는 후라이드 치킨을 좋아해 하지만 양념 치킨을 좋아해"]

In [1]:
import pandas as pd
import numpy as np

In [27]:
#tokenize
doc_ls=[doc.split() for doc in docs]
doc_ls

[['나는', '양념', '치킨을', '좋아해', '하지만', '후라이드', '치킨을', '싫어해'],
 ['나는', '후라이드', '치킨을', '좋아해', '하지만', '양념', '치킨을', '좋아해']]

In [28]:
word2id={}

index=0
for doc in doc_ls:
    for token in doc:
        if token not in word2id:
            word2id[token]=index
            index+=1
word2id

{'나는': 0, '양념': 1, '치킨을': 2, '좋아해': 3, '하지만': 4, '후라이드': 5, '싫어해': 6}

In [29]:
BoW_ls=[]

for doc in doc_ls:
    bow=np.zeros(len(word2id),dtype=int)
    for token in doc:
        bow[word2id[token]]+=1
    BoW_ls.append(bow.tolist())
BoW_ls

[[1, 1, 2, 1, 1, 1, 1], [1, 1, 2, 2, 1, 1, 0]]

In [30]:
from IPython.core import display as ICD

sorted_vocab=sorted((value,key) for key,value in word2id.items())
vocab=[v[1] for v in sorted_vocab]
for i in range(len(docs)):
    print("문서 {} : {}".format(i,docs[i]))
    ICD.display(pd.DataFrame([BoW_ls[i]],columns=vocab))
    print("\n")

문서 0 : 나는 양념 치킨을 좋아해 하지만 후라이드 치킨을 싫어해


,나는,양념,치킨을,좋아해,하지만,후라이드,싫어해
0,1,1,2,1,1,1,1




문서 1 : 나는 후라이드 치킨을 좋아해 하지만 양념 치킨을 좋아해


,나는,양념,치킨을,좋아해,하지만,후라이드,싫어해
0,1,1,2,2,1,1,0


In [31]:
#강사님 풀이_collections 사용
docs=["나는 양념 치킨을 좋아해 하지만 후라이드 치킨을 싫어해",
     "나는 후라이드 치킨을 좋아해 하지만 양념 치킨을 좋아해"]

doc_ls=[doc.split() for doc in docs]
doc_ls

from collections import defaultdict
word2id = defaultdict(lambda : len(word2id)) #word2id[token] 값 없으면 이걸로 들어감
[word2id[token] for doc in doc_ls for token in doc] #순서대로 도는 이중 for문
word2id

defaultdict(<function __main__.<lambda>()>,
            {'나는': 0,
             '양념': 1,
             '치킨을': 2,
             '좋아해': 3,
             '하지만': 4,
             '후라이드': 5,
             '싫어해': 6})

In [32]:
BoW_ls=[]

for doc in doc_ls:
    bow=np.zeros(len(word2id),dtype=int)
    for token in doc:
        bow[word2id[token]]+=1
    BoW_ls.append(bow.tolist()) #bow가 numpy array니까 list로 바꿔줌
BoW_ls

[[1, 1, 2, 1, 1, 1, 1], [1, 1, 2, 2, 1, 1, 0]]

gensim: nlp라이브러리(topic modelling for humans)

In [33]:
!pip install gensim

In [34]:
import gensim
from gensim import corpora 

docs=["나는 양념 치킨을 좋아해 하지만 후라이드 치킨을 싫어해",
     "나는 후라이드 치킨을 좋아해 하지만 양념 치킨을 좋아해"]

doc_ls=[doc.split() for doc in docs]
doc_ls

id2word = corpora.Dictionary(doc_ls)
BoW=[id2word.doc2bow(doc) for doc in doc_ls]
BoW
#(0,)(1,)~단어를 의미
#(,1)(,2)~단어 빈도수 표현

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1)],
 [(0, 1), (2, 1), (3, 2), (4, 2), (5, 1), (6, 1)]]

In [35]:
id2word.keys()

[0, 1, 2, 3, 4, 5, 6]

In [36]:
#단어사전 만들기
for key in id2word.keys():
    print("{} : {}".format(key, id2word[key]))

0 : 나는
1 : 싫어해
2 : 양념
3 : 좋아해
4 : 치킨을
5 : 하지만
6 : 후라이드


In [37]:
from gensim.matutils import sparse2full
#sparse2full: document,length줘야 함; gensim은 sparse한 형태->full은 벡터 형태로 바꿔주는 것
from IPython.core import display as ICD

vocab=[id2word[key] for key in id2word.keys()]
for i in range(len(docs)):
    print("문서 {} : {}".format(i,docs[i]))
    ICD.display(pd.DataFrame([sparse2full(BoW[i],len(vocab))],columns=vocab))
    print("\n")

문서 0 : 나는 양념 치킨을 좋아해 하지만 후라이드 치킨을 싫어해


,나는,싫어해,양념,좋아해,치킨을,하지만,후라이드
0,1.0,1.0,1.0,1.0,2.0,1.0,1.0




문서 1 : 나는 후라이드 치킨을 좋아해 하지만 양념 치킨을 좋아해


,나는,싫어해,양념,좋아해,치킨을,하지만,후라이드
0,1.0,0.0,1.0,2.0,2.0,1.0,1.0


TDM만들기

In [38]:
#차원=shape
#위의 TDM구조: 7*2 차원으로 만들기

TDM=np.zeros((len(word2id), len(doc_ls)), dtype=int)
TDM

array([[0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]])

In [39]:
for i,doc in enumerate(doc_ls):
    for token in doc:
        TDM[word2id[token],i] +=1
TDM #하나의 매트릭스에 단어와 단어 빈도수 표시 가능, DTM은 순서만 바꾸면 됨

array([[1, 1],
       [1, 1],
       [2, 2],
       [1, 2],
       [1, 1],
       [1, 1],
       [1, 0]])

In [43]:
#column명 만들기; 문서 기준
doc_names =['문서'+str(i) for i in range(len(doc_ls))]
sorted_vocab=sorted((value,key) for key,value in word2id.items())
vocab = [v[1] for v in sorted_vocab]
df_TDM=pd.DataFrame(TDM,columns=doc_names)
df_TDM['단어'] = vocab #vocab을 단어라는 column에 넣음
df_TDM.set_index('단어', inplace=True)
df_TDM #이걸로 tfidf 계산

,문서0,문서1
단어,,
나는,1,1
양념,1,1
치킨을,2,2
좋아해,1,2
하지만,1,1
후라이드,1,1
싫어해,1,0


In [51]:
#sklearn으로 TDM 만들기(주피터는 기본 DTM)
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
DTM = count_vect.fit_transform(docs)
DTM.toarray()

array([[1, 1, 1, 1, 2, 1, 1],
       [1, 0, 1, 2, 2, 1, 1]], dtype=int64)

In [53]:
doc_names =['문서'+str(i) for i in range(len(doc_ls))]
vocab = count_vect.get_feature_names_out()
df_TDM = pd.DataFrame(DTM.toarray().T, columns=doc_names) #.T = transpose
df_TDM['단어'] = vocab
df_TDM.set_index('단어', inplace=True)
df_TDM

,문서0,문서1
단어,,
나는,1,1
싫어해,1,0
양념,1,1
좋아해,1,2
치킨을,2,2
하지만,1,1
후라이드,1,1


TF-IDF = 단어빈도*다른 문서 등장횟수 역수

In [98]:
docs = ["오늘 동물원에서 원숭이를 봤어", 
       "오늘 동물원에서 코끼리를 봤어 봤어",
       "동물원에서 원숭이에게 바나나를 줬어 바나나를"]

In [99]:
doc_ls=[doc.split() for doc in docs]
doc_ls

[['오늘', '동물원에서', '원숭이를', '봤어'],
 ['오늘', '동물원에서', '코끼리를', '봤어', '봤어'],
 ['동물원에서', '원숭이에게', '바나나를', '줬어', '바나나를']]

In [147]:
from collections import defaultdict

word2id=defaultdict(lambda : len(word2id)) #defaultdict정의: key값이 없으면 value를 여기에 정의된 함수로 넣음
print(word2id)
[word2id[token] for doc in doc_ls for token in doc] #위의 이중for문 한 줄로 정리
print(word2id)

defaultdict(<function <lambda> at 0x0000013F1B0BE798>, {})
defaultdict(<function <lambda> at 0x0000013F1B0BE798>, {'오늘': 0, '동물원에서': 1, '원숭이를': 2, '봤어': 3, '코끼리를': 4, '원숭이에게': 5, '바나나를': 6, '줬어': 7})


In [148]:
DTM=np.zeros((len(doc_ls),len(word2id)), dtype=int)
DTM

array([[0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

In [149]:
for i,doc in enumerate(doc_ls):
    for token in doc:
        DTM[i,word2id[token]] +=1
DTM

array([[1, 1, 1, 1, 0, 0, 0, 0],
       [1, 1, 0, 2, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 2, 1]])

In [154]:
#각 단어에 대한 TF 구하기
#TF와 DTM 구조 똑같음
TF =np.zeros((len(doc_ls),len(word2id)))

doc_len = len(doc_ls)
word_len = len(word2id)


for i in range(doc_len):
    sum_doc=DTM[i].sum()
    for j in range(word_len):
        TF[i,j] = DTM[i,j] / sum_doc

TF

array([[0.25, 0.25, 0.25, 0.25, 0.  , 0.  , 0.  , 0.  ],
       [0.2 , 0.2 , 0.  , 0.4 , 0.2 , 0.  , 0.  , 0.  ],
       [0.  , 0.2 , 0.  , 0.  , 0.  , 0.2 , 0.4 , 0.2 ]])

In [155]:
#IDF 구하기
#count_nonzero; nonzero 개수 세기
#DTM[:,열] = 모든 행에 대하여 해당하는 열만 가져오겠다 ex. DTM[:,3] ~ 모든 행 중 3열에 해당하는 것 가져오기
import math

IDF = np.zeros(word_len)
for i in range(word_len):
    IDF[i] = -math.log10(np.count_nonzero([DTM[:,i]]) / doc_len)
IDF

array([ 0.17609126, -0.        ,  0.47712125,  0.17609126,  0.47712125,
        0.47712125,  0.47712125,  0.47712125])

In [156]:
#TFIDF 구하기
TFIDF = np.zeros(TF.shape)
TFIDF

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [157]:
for i in range(doc_len):
    for j in range(word_len):
        TFIDF[i,j] = TF[i,j] * IDF[j]
TFIDF
#문서1 키워드: 원숭이를
#문서2 키워드: 코끼리를
#문서3 키워드: 바나나를

array([[ 0.04402281, -0.        ,  0.11928031,  0.04402281,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.03521825, -0.        ,  0.        ,  0.0704365 ,  0.09542425,
         0.        ,  0.        ,  0.        ],
       [ 0.        , -0.        ,  0.        ,  0.        ,  0.        ,
         0.09542425,  0.1908485 ,  0.09542425]])